In [ ]:
!git clone https://github.com/woutervwessel/duplicateDetection.git new

Cloning into 'new'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
import pandas as pd
import random as rd
import json
import numpy as np
from itertools import combinations
import sympy as sy
from sklearn.cluster import AgglomerativeClustering
import re

In [ ]:
#Opening the .json which is read as a dictionary by python
with open('new/TVs-all-merged.json') as f:
    data = json.load(f)
print(type(data))

<class 'dict'>


In [ ]:
#Reshaping data to seperate elements and printing number of observations in full data set
new_data = {}
i = 1
for key in data.keys():
    for description in data[key]:
        new_data[i] = description
        i+=1
print(len(new_data.keys()))

1624


In [ ]:
#create set of titles 
def createTitleSet(data, includeKVP = False, onlyModelID = True):
  """Returns list of models words from the titles and feature map of each product
  param: data, dictionary with all product information.
  
  The cleaning works as follows:
  We attach all features in the title
  """
  if includeKVP == True:
      for i in range(1,len(data)):
        kvp = ' '
        for key in data[i]['featuresMap'].keys():
          kvp = kvp + ' ' + data[i]['featuresMap'][key]
        data[i]['title'] = data[i]['title']+kvp

  set_titles = {}
  for key in data.keys():
      set_titles[key] = data[key]['title']

  #Define regex functions
  regex = '[a-zA-Z]+[0-9]+[a-zA-Z0-9]*|[a-zA-Z0-9]*[0-9]+[a-zA-Z]+|[0-9]+[.][0-9]+[a-zA-Z]*'
  modelID_regex = '[a-zA-Z0-9]*[0-9]+[a-zA-Z]+[0-9]+[a-zA-Z0-9]*|[a-zA-Z0-9]*[a-zA-Z]+[0-9]+[a-zA-Z]+[a-zA-Z0-9]+'
  
  #Clean data
  new_title= set_titles.copy()
  for key in new_title.keys():
      new_title[key] = new_title[key].lower()
      new_title[key] = new_title[key].replace(' inches','inch')
      new_title[key] = new_title[key].replace(' inch','inch')
      new_title[key] = new_title[key].replace('"','inch')
      new_title[key] = new_title[key].replace('" ','inch ')
      new_title[key] = new_title[key].replace('hertz','hz')
      new_title[key] = new_title[key].replace(' hertz','hz')
      new_title[key] = new_title[key].replace(' hz','hz')
      new_title[key] = re.sub("[^a-zA-Z0-9\s\.]","",new_title[key])
      
      #Maintain only modelID formatted words as proposed in the paper if onlyModelID == True
      if onlyModelID == True:
            modelID_title = re.findall(modelID_regex, new_title[key])
            if not modelID_title:
                new_title[key] = re.findall(regex, new_title[key]) 
                new_title[key] = list(set(new_title[key]))
            else:
                new_title[key] = list(set(modelID_title))
      #If onlyModelID == False, we maintain all model words from the titles in the model word list
      else:
            new_title[key] = re.findall(regex, new_title[key]) 
            new_title[key] = list(set(new_title[key])) 

  return new_title

In [ ]:
def createBinaryMatrix(data):
    """
    param: data, dictionary, length=|P| 
    param: set_titles, dictionary, lenght=|P|
    Returns binary values, list, shape=(|MW|, |P|)
    """
    
    #Create model word list
    data_conc = sum(data.values(), [])
    MW = sorted(set(data_conc), key=data_conc.index) #extracts unique model words 

    #create binary vector 
    #values are set to 1 if model word is present in title
    binary_matrix = np.zeros((len(MW),len(data)),dtype=int)
    p=0
    for value in data.values():
        w=0
        for word in MW:
            if word in value:
                binary_matrix[w,p]=1
            w= w+1
        p=p+1
    return binary_matrix

In [ ]:
#MinHash algorithm
def createHashValues(lengthBinary, numHashes):
  """
  This function creates hash values by permutating lengthBinary rows, numHashes times, so that we obtain a hash matrix of size lengthBinary times numHashes
  param: lengthBinary, integer = |new_title|
  param: numHashes, integer number of hash function
  """
  mylist = []
  hashval = list(range(1,lengthBinary+1))
  hashind = list(range(1,lengthBinary+1))
  for i in range(numHashes):
    rd.shuffle(hashind)
    dicto = dict(zip(hashind,hashval))
    mylist.append(dicto)
  return mylist
  
def createSignature(binary_matrix,hashValues,numHashes):
  """
  Returns a list in list containing signatures for each product (Size: #Products x #numHashes)
  """
  signatures=[]
  for product in range(binary_matrix.shape[1]):
    sig_val = []
    for i in range(numHashes): #N signature values
      for hash_val in range(len(binary_matrix)): #Find minHashValue for i-th signature value, goes from 0 to 1299
        #print(product,i,hash_val)
        index =  hashValues[i].get(hash_val+1) #add 1 to hash value as we go from 1 to 1300 and not 0 to 1299
        if (binary_matrix[index-1,product]) == 1:
          sig_val.append(hash_val+1)
          break
    signatures.append(sig_val)
  return signatures

def minHashing(binary_matrix, numHashes):
  """
  Returns signatures for each product
  """
  hashValues = createHashValues(len(binary_matrix),numHashes)
  signatures = createSignature(binary_matrix,hashValues,numHashes)
  return signatures

In [ ]:
#LSH
def hashSignaturesToBuckets(data, signatures, b):
    """
    Hashes column bands of each signature to a buckets of that band 
    : param signatures: array-like, shape=(|P|, n)
    returns dictionary with buckets for each band 
    """
    n = len(signatures[0])
    assert n % b == 0  
    r = int(n/b)   #number of rows in each band
    product_keys = list(data.keys()) #to get right product key with index
    
    bucket_bands =[]
    for i in range(b): #construct for each band a empty list of buckets 
        bucket_bands.append({})

    buckets=[]
    sign_idx=0

    #goes trough all product signatures
    for signature in signatures: 
        bands= signatureToBands(signature, r)

        #goes trough each band i of a product signature
        for i, band in enumerate(bands.astype(str)): 
            band = ','.join(band)
            if band not in bucket_bands[i].keys():
                bucket_bands[i][band]=[]
            bucket_bands[i][band].append(product_keys[sign_idx])
        sign_idx +=1
    return bucket_bands


def signatureToBands(signature, r):
    """
    Creates bands of length r for each signature vector
    : param signature: array-like, shape=(1, |P|)
    b: number of bands 
    returns list of subvectors for signature
    """
    bands=[]
    for i in range(0,len(signature), r):#step size r 
        bands.append(signature[i:i+r])
    return np.stack(bands)


def candidates(bucket_bands):
    """
    Returns list of candidate neigbors
    """
    candidates = []
    for bucket_band in bucket_bands:
        for bucket in bucket_band.keys():
            products_in_bucket = bucket_band[bucket]
            if len(products_in_bucket) > 1:
                candidates.extend(combinations(products_in_bucket, 2))
    return list(set(candidates)) #returns unique pairs 

In [ ]:
def SSM(candidate_pairs, new_title, sim_tres):
    """
    SSM calculates the similarity between two candidate product pairs produced by LSH
    It computes the percentage of matching modelwords in the title
    When this is above threshold (sim_tres), a pair is seen as duplicate
    All pairs that are seen as duplicates are returned as a list called found_pairs
    """
    found_pairs = [];
    for i in range(len(candidate_pairs)):
        prod1 = candidate_pairs[i][0]
        prod2 = candidate_pairs[i][1]
        simscore = len(list(set(new_title[prod1]).intersection(set(new_title[prod2]))))/len(list(set(new_title[prod1]).union(set(new_title[prod2]))))
        if simscore > sim_tres:
            found_pairs.append(candidate_pairs[i])
    return found_pairs

In [ ]:
def sampleData(data, n_samples):
  """Samples data
  param: data, dictionary with all product information, dictionary
  param: n_samples, number of samples to draw, integer 
  """
  sample_d = {}
  for i in range(1, n_samples+1):
    draw = rd.randint(1,len(data))
    sample_d[i] = data[draw]
  return sample_d

def truePositive(candidate_pairs):
  """Return number of true postive pairs
  param: candidate_pairs, list of canidates pairs obtained from LSH"""
  TP=0
  for candidate_pair in candidate_pairs:
    if data[candidate_pair[0]]['modelID']==data[candidate_pair[1]]['modelID']:
      TP +=1
  return TP

def totalAmountDuplicates(data):
  """Returns number of duplicates in the data"""
  modelIDs = {}
  for key in data.keys():
    modelIDs[key] = data[key]['modelID']

  rev_dict = {} #dictionary with key: ModelID and value: {indices with that modelID}
  for key, modelID in modelIDs.items():
    rev_dict.setdefault(modelID, set()).add(key)

  duplicates = []
  for value in rev_dict.values():
    if len(value) > 1:
      duplicates.extend(combinations(value, 2))

  return duplicates


In [ ]:
def computeMeasures(duplicates, candidate_pairs, found_pairs):
    """Returns PQ, PC, F1* and F1"""
    #make sets from results to apply intersection/union functions
    duplicates = set(duplicates)
    candidate_pairs = set(candidate_pairs)
    found_pairs = set(found_pairs)

    total_duplicates = len(duplicates)

    #measures LSH
    total_duplicates_found = len(candidate_pairs)
    Df = len(candidate_pairs.intersection(duplicates))

    #measures MSM
    TP = len(found_pairs.intersection(duplicates))
    FP = len(found_pairs) - TP
    FN = total_duplicates - TP

    #metrics
    PQ = Df / total_duplicates_found
    PC = Df / total_duplicates
    F1_star = (2 * PQ * PC)/ (PQ+PC)
    precision = TP / len(found_pairs)
    recall = TP / (TP + FN)
    F1 = (2 * recall * precision ) / (recall + precision)
  

    return PQ, PC, F1_star, F1

### Perform bootstrap and compute evaluation metrics


In [ ]:
#Bootstraps for LSH performance
bootstraps = 5
iter = 0
settings =[]
T = []

n_samples = int(len(new_data)*0.6) #
n = 720 #number of hash functions for signature matrix
sim_tres = 0.6 #similarity treshold for SSM

for r in [1,2,3,4,5,6,8,10,15,20,30,48]: #Manually configured based on threshold values
    if (n % r) == 0:
        b = n/ r
        t = (1/b) ** (1/r)
        settings.append([t,b,r])
        T.append(t)
        

while iter != bootstraps:
    #Sample data
    data = sampleData(new_data, n_samples)
    
    #Create titles
    set_titles = createTitleSet(data, includeKVP = False, onlyModelID = False)

    #Start minhasing
    binary_matrix = createBinaryMatrix(set_titles) 
    signatures = minHashing(binary_matrix, n)
    
    duplicates  = totalAmountDuplicates(data)
    
    #Metric values for all band values for bootstrap i 
    PQ_i  = [] 
    PC_i = []
    F1star_i = []
    F1_i = []
    foc_i = []

    for setting in settings:
        #Apply LSH
        bands =  int(setting[1])
        bucket_bands = hashSignaturesToBuckets(set_titles, signatures, bands)
        candidate_pairs= candidates(bucket_bands)
        
        #Apply SSM
        found_pairs = SSM(candidate_pairs, set_titles, sim_tres)
        
        #Compute measures
        PQ, PC, F1_star, F1 = computeMeasures(duplicates, candidate_pairs, found_pairs)
        fraction_of_comparison = len(candidate_pairs) / len(list(combinations(data.keys(), 2)))
     
        PQ_i.append(PQ)
        PC_i.append(PC)
        F1star_i.append(F1_star)
        F1_i.append(F1)
        foc_i.append(fraction_of_comparison)
        
    if iter==0:
        measures = np.stack((PQ_i, PC_i, F1star_i, F1_i, foc_i), 0)
    else:
        measures += np.stack((PQ_i, PC_i, F1star_i, F1_i, foc_i), 0)
        
    iter += 1

#averaged measures with rows=(PQ, PC, F1star, F1) and columns=|settings|
av_measures = measures / bootstraps
print('\n PQ:', av_measures[0], '\n PC:', av_measures[1], '\n F1_star', av_measures[2], '\n F1', av_measures[3], '\n fraction_of_comparisons', av_measures[4])


 PQ: [0.00059486 0.00068616 0.00352273 0.015061   0.0647036  0.16996872
 0.31474104 0.40203562 0.47590361 0.48615385 0.51633987 0.51815182] 
 PC: [0.50704225 0.50704225 0.47183099 0.42018779 0.39201878 0.38262911
 0.37089202 0.37089202 0.37089202 0.37089202 0.37089202 0.3685446 ] 
 F1_star [0.00118832 0.00137047 0.00699325 0.02907968 0.11107416 0.23537906
 0.34051724 0.38583639 0.41688654 0.4207723  0.43169399 0.43072702] 
 F1 [0.39303483 0.39303483 0.39303483 0.39303483 0.39303483 0.39303483
 0.395      0.39949431 0.41688654 0.4207723  0.43169399 0.43072702] 
 fraction_of_comparisons [7.66297845e-01 6.64333303e-01 1.20413379e-01 2.50817240e-02
 5.44685988e-03 2.02384294e-03 1.05940475e-03 8.29374635e-04
 7.00642185e-04 6.85869609e-04 6.45772616e-04 6.39441512e-04]
